In [136]:
import requests
import bs4 as bs
import lxml
httpaddr = "https://www.clarin.com"

requests_ret = requests.get(httpaddr)
print(requests_ret, end = ' || ')
soup = bs.BeautifulSoup(requests_ret.text, 'lxml')
# Notas:
# Everything semmantically a header, is tagged with <h2>. To all scrapping practical
# purposes, all headers except h2 are deemded irrelevant
articles = soup.find_all("article"); print("articles :", len(articles), end = ' / ')
sections = soup.find_all("section"); print("sections :", len(sections), end = ' / ')
h1s = soup.find_all("h1"); print("h1s :", len(h1s), end = ' / ')
h2s = soup.find_all("h2"); print("h2s :", len(h2s), end = ' / ')
h3s = soup.find_all("h3"); print("h3s :", len(h3s), end = ' / ')
h4s = soup.find_all("h4"); print("h4s :", len(h4s), end = ' / ')
h5s = soup.find_all("h5"); print("h5s :", len(h5s), end = ' / ')
h6s = soup.find_all("h6"); print("h6s :", len(h6s))
# only <h2>s hold relevance
# the only <h1> holds an SVG graphic

for discard_tag in ("script", "style"):
    for t in soup.find_all(discard_tag): t.extract()


<Response [200]> || articles : 117 / sections : 21 / h1s : 1 / h2s : 146 / h3s : 0 / h4s : 0 / h5s : 0 / h6s : 0


In [137]:
# DUMPS A REPORT ON THE NESTING OF <article>s IN <section>s:
# TODO: This can be generalized as a routine that, for any two lists of nodes, gives a report on nesting

# Check for section nesting
# There's an almost strict hierarchical nesting in which articles are always inside some section
# The double belonging of to sections 2 and 3 are due to the fact that section 2 is nested inside section 3 (July 24, 2022)
print("+++ <section> NESTING CHECK :: ")
for (i, j) in [(i, j) for (i, section_i) in enumerate(sections) for (j, section_j) in enumerate(sections) if section_j in section_i.descendants]:
    print(("\t<section>[%" + str(len(str(len(sections)))) + "d] is nested in <section>[%" + str(len(str(len(sections)))) + "d]") % (j, i))

print("+++ <article> NESTING CHECK :: ")
for (i, j) in [(i, j) for (i, article_i) in enumerate(articles) for (j, article_j) in enumerate(articles) if article_j in article_i.descendants]:
    print(("\t<article>[%" + str(len(str(len(articles)))) + "d] is nested in <article>[%" + str(len(str(len(articles)))) + "d]") % (j, i))

# dangling <article>s
# these, the <article>s not-in-any-section, are the articles depicted as "LO MAS VISTO POR LOS SUSCRIPTORES" at the tail of the page
articles_not_in_sections = articles[:]              # copy articles ...
articles_in_sections_set = set()
for section in sections:
    for article in articles:
        if article in section.descendants:
            articles_in_sections_set = articles_in_sections_set | {article} # need to use this notation. set() notation consumes an interable and therefore iterates over article's members
for article in articles_in_sections_set:
    while article in articles_not_in_sections:      # while instead of if, just in case of repetition
        articles_not_in_sections.remove(article)    # ... then substract every article already under a <section>
print("+++ NUMBER OF <article>s NOT UNDER ANY <section> :", len(articles_not_in_sections))

# Report which <article> is nested in which <section>s
maxlen = max( (len('section[' + 'x' * len(str(len(sections))) + ']:' + str(section.attrs['class'])) for section in sections) )
for i, section in enumerate(sections):
    line = (('section[%' + str(len(str(len(sections)))) + 'd]: %s' + ' ' * 100) % (i, str(section.attrs['class']),))[:maxlen] + " ||"
    for article in articles: line += "X" if article in section.descendants else '.'; 
    line += "|"
    print(line)

+++ <section> NESTING CHECK :: 
	<section>[ 3] is nested in <section>[ 2]
+++ <article> NESTING CHECK :: 
+++ NUMBER OF <article>s NOT UNDER ANY <section> : 6
section[ 0]: ['container', 'no-p']                                         ||XXXXX................................................................................................................|
section[ 1]: ['container', 'no-p', 'listado']                              ||.....XXXXXXXXXXXXXX..................................................................................................|
section[ 2]: ['container', 'listado', 'no-p']                              ||...................XXXXXXXXXXXX......................................................................................|
section[ 3]: ['columnas']                                                  ||...................XXXXXX............................................................................................|
section[ 4]: ['container', 'listado', 'no-p']            

In [138]:
articles_by_section = [[] for _ in sections]    # ... beware of reference replication...
for i, section in enumerate(sections):
    for j, article in enumerate(articles):
        if article in section.descendants: 
            articles_by_section[i].append(article)

# for i, art_by_sect in enumerate(articles_by_section): print(("<section>[%0" + str(len(str(len(sections)))) + "d]") % (i, ), "contains", len(art_by_sect), "articles")

articles_by_section_dict = {i : l for (i, l) in enumerate(articles_by_section)}
articles_by_section_dict.update({'none' : articles_not_in_sections})
# for k in articles_by_section_dict:
#     print(("For articles in <section>[%" + str(max([len(str(k)) for k in articles_by_section_dict])) + "s] number of articles:") % k, len(articles_by_section_dict[k]))

idx_width = max([len(str(k)) for k in articles_by_section_dict])
val_width = max([len(str(len(articles_by_section_dict[k]))) for k in articles_by_section_dict])
for i, k in enumerate(articles_by_section_dict):
    print(("<section>[%" + str(idx_width) + "s] " + "= %" + str(val_width) + "s") % 
          (k, str(len(articles_by_section_dict[k]))), end = '\n' if (i + 1) % 6 == 0 else ' // ')


<section>[   0] =  5 // <section>[   1] = 14 // <section>[   2] = 12 // <section>[   3] =  6 // <section>[   4] =  7 // <section>[   5] =  4
<section>[   6] =  5 // <section>[   7] =  4 // <section>[   8] =  6 // <section>[   9] =  6 // <section>[  10] =  6 // <section>[  11] =  6
<section>[  12] =  6 // <section>[  13] =  3 // <section>[  14] =  1 // <section>[  15] =  6 // <section>[  16] =  4 // <section>[  17] =  4
<section>[  18] =  4 // <section>[  19] =  4 // <section>[  20] =  4 // <section>[none] =  6 // 

In [139]:
# dumping some data by article as classified in articles_by_section_dict
for k, a_list in articles_by_section_dict.items():
    for j, article in enumerate(a_list):
        
        h2ss = article.find_all('h2') if article.find_all('h2') else None
        
        volanta = article.find(class_ = 'volanta') if article.find(class_ = 'volanta') else ''
        author = article.find(class_ = 'author') if article.find(class_ = 'author') else ''
        author_rel = article.find(class_ = 'author-rel') if article.find(class_ = 'author-rel') else ''
        summary = article.find(class_ = 'summary') if article.find(class_ = 'summary') else ''
        section = article.find(class_ = 'section') if article.find(class_ = 'section') else ''
        section_rel = article.find(class_ = 'section-rel') if article.find(class_ = 'section-rel') else ''
        
        print("<section>[%s] / <article>[%d]" % (str(k), j))
        print('-' * len("<section>[%s] / <article>[%d]" % (str(k), j)))
        if len(h2ss) != 1:
            if len(h2ss) > 1:
                print('\t++++++++MORE THAN ONE <h2>!!!++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
            else:
                print('\t++++++++NO <h2>!!!+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
        for i, h in enumerate(h2ss):
            print('\th2[%d] = %s' % (i, h))
#         print( '\tvolanta = "%s"' % (volanta, ),
#                '\n\tauthor = "%s"' % (author, ),
#                '\n\tsummary = "%s"' % (summary, ),
#                '\n\tsection = "%s"' % (section, ))
#         if volanta : print('\tvolanta = "%s"' % (volanta.strip()[:60], ))
#         if author : print('\tauthor = "%s"' % (author.strip()[:60], ))
#         if summary : print('\tsummary = "%s"' % (summary.strip()[:60], ))
#         if section : print('\tsection = "%s"' % (section.strip()[:60], ))
        if volanta : print('\tvolanta = "%s"' % (str(volanta)[:80], ))
        if author : print('\tauthor = "%s"' % (str(author)[:80], ))
        if author_rel : print('\tauthor = "%s"' % (str(author_rel)[:80], ))
        if summary : print('\tsummary = "%s"' % (str(summary)[:80], ))
        if section : print('\tsection = "%s"' % (str(section)[:80], ))
        if section_rel : print('\tsection_rel = "%s"' % (str(section_rel)[:80], ))
        print()
    print('-' * 60)

<section>[0] / <article>[0]
---------------------------
	h2[0] = <h2>Guiño de Massa al campo: habló con el presidente de La Sociedad Rural y se reunirá con la Mesa de Enlace</h2>
	volanta = "<p class="volanta">Antes de asumir </p>"
	author = "<p class="author">Silvia Naishtat</p>"
	summary = "<p class="summary">Nicolás Pino se comunicó con el flamante ministro de Economía"

<section>[0] / <article>[1]
---------------------------
	h2[0] = <h2 class="title-rel">El campo quiere ser parte de la solución pero la política se interpone</h2>
	author = "<span class="author-rel">Héctor A. Huergo</span>"
	section_rel = "<span class="section-rel">Política.</span>"

<section>[0] / <article>[2]
---------------------------
	h2[0] = <h2>Massa define los nombres para el nuevo ministerio y un plan para los primeros diez días</h2>
	volanta = "<p class="volanta">La nueva etapa</p>"
	section = "<p class="section">Política</p>"

<section>[0] / <article>[3]
---------------------------
	h2[0] = <h2>Duro discu

In [140]:
# dump source under every single <article>:
for i, article in enumerate(articles_by_section[0]):
    print("<article>[%d] : %s" % (i, len(article.parent.find_all('li', class_ = 'item-related'))))
    print("-------------")
    
    print(article.parent.prettify())
    print('-' * 60)

<article>[0] : 1
-------------
<li class="m-box child h300 col-lg-6 col-md-8 col-sm-12 col-xs-12 no-p m2x1" data-classes="m-box child m2x1 h300  col-lg-6 col-md-8 col-sm-12 col-xs-12 no-p" data-name="content" data-size-h="1" data-size-w="2">
 <article class="content-nota list-format twoxone_no_foto">
  <div class="mt">
   <div class="max-h flex-change">
    <h2>
     Guiño de Massa al campo: habló con el presidente de La Sociedad Rural y se reunirá con la Mesa de Enlace
    </h2>
    <p class="volanta">
     Antes de asumir
    </p>
   </div>
   <div class="data-txt">
    <p class="author">
     Silvia Naishtat
    </p>
   </div>
   <p class="summary">
    Nicolás Pino se comunicó con el flamante ministro de Economía y le remarcó la importancia de que Agricultura no deje de ser ministerio. Los nombres en danza para ocupar el cargo que dejó Julián Domínguez.
   </p>
  </div>
  <a aria-label="/economia/dialogo-whatsapp-massa-presidente-rural_0_Mj5doSJqB9.html" class="link_article" href="

In [141]:
# How many <li>s are on each <article>?

for i, article in enumerate(articles):
    print("<article>[%3s]" % i, str(len(list(article.parent.find_all('li')))) if len(list(article.parent.find_all('li'))) != 0 else "", 
          end = ' ' if (i+1) % 8 != 0 else '\n')

<article>[  0] 1 <article>[  1]  <article>[  2]  <article>[  3]  <article>[  4]  <article>[  5]  <article>[  6]  <article>[  7] 
<article>[  8]  <article>[  9]  <article>[ 10]  <article>[ 11]  <article>[ 12]  <article>[ 13]  <article>[ 14]  <article>[ 15] 
<article>[ 16]  <article>[ 17]  <article>[ 18]  <article>[ 19]  <article>[ 20]  <article>[ 21]  <article>[ 22]  <article>[ 23] 
<article>[ 24]  <article>[ 25]  <article>[ 26]  <article>[ 27]  <article>[ 28]  <article>[ 29]  <article>[ 30]  <article>[ 31] 
<article>[ 32]  <article>[ 33]  <article>[ 34]  <article>[ 35]  <article>[ 36]  <article>[ 37]  <article>[ 38]  <article>[ 39] 
<article>[ 40]  <article>[ 41]  <article>[ 42]  <article>[ 43]  <article>[ 44]  <article>[ 45]  <article>[ 46]  <article>[ 47] 
<article>[ 48]  <article>[ 49]  <article>[ 50]  <article>[ 51]  <article>[ 52]  <article>[ 53]  <article>[ 54]  <article>[ 55] 
<article>[ 56]  <article>[ 57]  <article>[ 58]  <article>[ 59]  <article>[ 60]  <article>[ 61]  <articl

In [142]:
for i, art in enumerate(articles_by_section_dict[1][7:]):
    print(i, '\n', art.parent.parent.parent)

0 
 <div class="last-moment" id="lmvmod_202206596043799" xmlns="http://www.w3.org/1999/html">
<a class="tag" href="https://www.clarin.com/ultimo-momento/" style="cursor: pointer;" target="_blank"></a>
<ul class="box-items">
<li aria-label="/mundo/crimen-nina-9-anos-apunalada-calle-conmociona-inglaterra_0_0vjiDBbhn3.html" onclick="window.open('/mundo/crimen-nina-9-anos-apunalada-calle-conmociona-inglaterra_0_0vjiDBbhn3.html'); metricsView('bloques_home','bloque_ultimo_momento|0vjiDBbhn3', true);" style="cursor: pointer;">
<article class="flex-change">
<h2>El crimen de una niña de 9 años apuñalada en la calle conmociona a Inglaterra</h2>
<span>19:53</span>
</article>
</li>
<li aria-label="/sociedad/san-juan-detuvieron-10-personas-intento-saqueo-conocido-supermercado_0_Lo5hQLtDqM.html" onclick="window.open('/sociedad/san-juan-detuvieron-10-personas-intento-saqueo-conocido-supermercado_0_Lo5hQLtDqM.html'); metricsView('bloques_home','bloque_ultimo_momento|Lo5hQLtDqM', true);" style="cursor

In [143]:
# treewalk a given node and dump source structure under that node's tag:
def dumptree(n, prefix):
    # global articles
    # global sections   
    stack = []
    stack.extend(reversed([ch for ch in n.children if type(ch) == bs.element.Tag]))     # Warning! Generators tend to explode...
    print(prefix + n.name + ' // ', end = '')
    try:
        if n.name == 'h2' : print(n.attrs, '/', n.get_text().strip()[:1000])
        elif n.name == 'p' : print(n.attrs, '/', n.get_text().strip()[:100])
        elif n.name == 'span' : print(n.attrs, '/', n.get_text().strip()[:100])
        elif  n.name == 'a' : print(n.attrs['href'][:100], '/', ' '.join(n.get_text().strip().split('\n')))
        elif  n.name == 'div' : print(n.attrs['class'][:100])
        elif  n.name == 'section':
            print(n.name + '[' + str(sections.index(n)) + ']')
        elif  n.name == 'article':
            print(n.name + '[' + str(articles.index(n)) + ']')
        else:
            print(n.name)
    except KeyError:
        print("+++ ERROR WHILE TRYING TO ACCESS NONEXISTANT KEY!!!+++")
    while stack:
        dumptree(stack.pop(), '|   ' + prefix)
    return None


i = 0
print("<section>[%02d]" % i)
print("-------------")
dumptree(sections[i].parent, '')


<section>[00]
-------------
div // ['container', 'father', 'no-p']
|   section // section[0]
|   |   ul // ul
|   |   |   li // li
|   |   |   |   article // article[0]
|   |   |   |   |   div // ['mt']
|   |   |   |   |   |   div // ['max-h', 'flex-change']
|   |   |   |   |   |   |   h2 // {} / Guiño de Massa al campo: habló con el presidente de La Sociedad Rural y se reunirá con la Mesa de Enlace
|   |   |   |   |   |   |   p // {'class': ['volanta']} / Antes de asumir
|   |   |   |   |   |   div // ['data-txt']
|   |   |   |   |   |   |   p // {'class': ['author']} / Silvia Naishtat
|   |   |   |   |   |   p // {'class': ['summary']} / Nicolás Pino se comunicó con el flamante ministro de Economía y le remarcó la importancia de que Agr
|   |   |   |   |   a // /economia/dialogo-whatsapp-massa-presidente-rural_0_Mj5doSJqB9.html / 
|   |   |   |   ul // ul
|   |   |   |   |   li // li
|   |   |   |   |   |   article // article[1]
|   |   |   |   |   |   |   a // /politica/campo-quiere

In [146]:
parent_lis = []
gparent_uls = []
ggparent_uls = []
gggparent_uls = []
ggggparent_uls = []

for article in articles:
    if article.parent.name == 'li':
        parent_lis.append(article.parent)
        gparent_uls.append(article.parent.parent)
        ggparent_uls.append(article.parent.parent.parent)
        gggparent_uls.append(article.parent.parent.parent.parent)
        ggggparent_uls.append(article.parent.parent.parent.parent.parent)

i = 5
for s in "parent_lis", "gparent_uls", "ggparent_uls", "gggparent_uls", "ggggparent_uls":
    exec(f'dumptree({s}[{i}], "")')
    print()

li // li
|   article // article[5]
|   |   div // ['mt']
|   |   |   div // ['flex-change']
|   |   |   |   h2 // {} / La interna K golpea a Máximo en Diputados y crecen las críticas en su propio bloque
|   |   |   |   p // {'class': ['volanta']} / En medio de la crisis oficial
|   |   |   div // ['data-txt']
|   |   |   |   p // {'class': ['author']} / Gustavo Berón
|   |   div // ['wrap-figure']
|   |   |   picture // picture
|   |   |   |   source // source
|   |   |   |   img // img
|   |   a // /politica/interna-k-golpea-maximo-kirchner-diputados-crecen-criticas-bloque_0_pKpzuX6J0v.html / 

ul // ul
|   li // li
|   |   article // article[5]
|   |   |   div // ['mt']
|   |   |   |   div // ['flex-change']
|   |   |   |   |   h2 // {} / La interna K golpea a Máximo en Diputados y crecen las críticas en su propio bloque
|   |   |   |   |   p // {'class': ['volanta']} / En medio de la crisis oficial
|   |   |   |   div // ['data-txt']
|   |   |   |   |   p // {'class': ['author']} / 